In [7]:
import json
import pandas as pd
import numpy as np
from copy import copy

with open("/opt/ml/segmentation/input/data/train_all_revise.json", "r") as f:
    train = json.load(f)
split_train = copy(train)
split_valid = copy(train)
df = pd.DataFrame(train['annotations'])

In [8]:
df.tail()

,id,image_id,category_id,segmentation,area,bbox,iscrowd
26229,26397,3270,2,"[[0, 180, 0, 180, 0, 179, 0, 178, 0, 177, 0, 1...",1451.5,"[0.0, 97.2, 28.9, 82.9]",0
26230,26398,3270,2,"[[0, 175, 0, 175, 0, 174, 0, 173, 0, 172, 0, 1...",212.5,"[0.0, 151.4, 17.7, 24.1]",0
26231,26399,3270,2,"[[5, 408, 4, 408, 3, 408, 2, 408, 1, 408, 0, 4...",14152.5,"[0.0, 261.0, 112.8, 147.7]",0
26232,26400,3270,2,"[[7, 511, 6, 511, 5, 511, 4, 511, 3, 511, 2, 5...",1044.0,"[0.3, 426.3, 16.7, 85.6]",0
26233,26401,3271,4,"[[314, 430, 313, 430, 312, 430, 311, 430, 310,...",27000.0,"[258.7, 158.3, 124.9, 272.0]",0


In [9]:
class_per_image = [(j, [0 for i in range(11)]) for j in range(3272)]
def clean(x):
    '''
    for apply : 
    '''

    class_per_image[x['image_id']][1][x['category_id']] += 1

df.apply(clean, axis=1)
class_per_image = np.array(class_per_image)

np.random.seed(2021)
np.random.shuffle(class_per_image)
class_per_image

/opt/conda/envs/segmentation/lib/python3.7/site-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # Remove the CWD from sys.path while we load stuff.


array([[406, list([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0])],
       [2926, list([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0])],
       [3030, list([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])],
       ...,
       [1152, list([0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0])],
       [2105, list([0, 6, 22, 0, 1, 0, 0, 0, 2, 0, 0])],
       [1140, list([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])]], dtype=object)

In [16]:
train_classes = np.zeros((11))
train_indices = []
valid_classes = np.zeros((11))
valid_indices = []

ratio = 0.15
for image_id, classes in (class_per_image):
    train_if_train = train_classes + classes
    valid_if_train = valid_classes / ratio
    mse_if_train = ((train_if_train - valid_if_train)**2).mean(axis=0)
    
    train_if_valid = train_classes
    valid_if_valid = (valid_classes + classes) / ratio
    mse_if_valid = ((train_if_valid - valid_if_valid)**2).mean(axis=0)

    ### Random Forcing
    if(np.random.random() < 0.2):
        '''
        Random Forcing이 없으면 한 image 안에 객체가 많을 경우 
        Validation set으로 넣어버리면 mse가 갑자기 높아지기 때문에
        그냥 train set으로 분류해버리는 경향이 있음
        '''
        valid_indices.append(int(image_id))
        valid_classes += classes
        continue
    
    if mse_if_train < mse_if_valid:
        train_indices.append(int(image_id))
        train_classes += classes
    else:
        valid_indices.append(int(image_id))
        valid_classes += classes

print("Train images :" , len(train_indices))
print("Validation images :" , len(valid_indices))
print('-'*60)
print("Distribution of classes in Train dataset")
print(train_classes)
print('-'*60)
print("Distribution of classes in Validation dataset")
print(valid_classes)

Train images : 2625
Validation images : 647
------------------------------------------------------------
Distribution of classes in Train dataset
[   0. 2280. 7481.  535.  458.  512. 2534. 1127. 6147.   44.  134.]
------------------------------------------------------------
Distribution of classes in Validation dataset
[   0.  493. 1832.  124.  104.   98.  557.  216. 1496.   18.   44.]


In [17]:
split_train['images']

[{'license': 0,
  'url': None,
  'file_name': 'batch_01_vt/0002.jpg',
  'height': 512,
  'width': 512,
  'date_captured': None,
  'id': 0},
 {'license': 0,
  'url': None,
  'file_name': 'batch_01_vt/0003.jpg',
  'height': 512,
  'width': 512,
  'date_captured': None,
  'id': 1},
 {'license': 0,
  'url': None,
  'file_name': 'batch_01_vt/0005.jpg',
  'height': 512,
  'width': 512,
  'date_captured': None,
  'id': 2},
 {'license': 0,
  'url': None,
  'file_name': 'batch_01_vt/0006.jpg',
  'height': 512,
  'width': 512,
  'date_captured': None,
  'id': 3},
 {'license': 0,
  'url': None,
  'file_name': 'batch_01_vt/0007.jpg',
  'height': 512,
  'width': 512,
  'date_captured': None,
  'id': 4},
 {'license': 0,
  'url': None,
  'file_name': 'batch_01_vt/0009.jpg',
  'height': 512,
  'width': 512,
  'date_captured': None,
  'id': 5},
 {'license': 0,
  'url': None,
  'file_name': 'batch_01_vt/0010.jpg',
  'height': 512,
  'width': 512,
  'date_captured': None,
  'id': 6},
 {'license': 0,
  'u

In [19]:
# train, valid에 해당하는 image/annotation을 각각 저장
train_images = []
train_ann = []
valid_images = []
valid_ann = []
for image in split_train['images']:
    if image['id'] in train_indices:
        train_images.append(image)
    else:
        valid_images.append(image)
for ann in split_train['annotations']:
    if ann['image_id'] in train_indices:
        train_ann.append(ann)
    else:
        valid_ann.append(ann)
split_train['images'] = train_images
split_train['annotations'] = train_ann
split_valid['images'] = valid_images
split_valid['annotations'] = valid_ann

len(split_train['images']), len(split_valid['images'])

(2625, 643)

In [20]:
# Save
with open("/opt/ml/segmentation/input/data/train_v1.json", "w") as f:
    json.dump(split_train, f, indent=4)
with open("/opt/ml/segmentation/input/data/valid_v1.json", "w") as f:
    json.dump(split_valid, f, indent=4)